In [ ]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import contractions
import inflect
import re, string, unicodedata
from joblib import dump, load
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

# Transformadores Personalizados
Inicialmente se crearon 2 transformadores con las operaciones personalizadas usadas en la solución propuesta. El primer transformer, llamado LimpiezaTransformer, se encarga de preparar los datos en cuanto a formato, validez de los caracteres, mayúsculas y minúsculas, puntuación y demás.

In [ ]:
class LimpiezaTransformer(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    
    def limpiar_study(self,value):
        return value.replace('study interventions are ', '')
    
    def remove_non_ascii(self,words):
        """Remove non-ASCII characters from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words

    def to_lowercase(self,words):
        """Convert all characters to lowercase from list of tokenized words"""
        new_words =[]
        for word in words:
            new_word = word.lower()
            new_words.append(new_word)
        return new_words

    def remove_punctuation(self,words):
        """Remove punctuation from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words

    def replace_numbers(self,words):
        """Replace all interger occurrences in list of tokenized words with textual representation"""
        p = inflect.engine()
        new_words = []
        for word in words:
            if word.isdigit():
                new_word = p.number_to_words(word)
                new_words.append(new_word)
            else:
                new_words.append(word)
        return new_words

    def remove_stopwords(self,words):
        """Remove stop words from list of tokenized words"""
        new_words = []
        stopwords = nltk.corpus.stopwords.words('english')
        for word in words:
            if word not in stopwords:
                new_words.append(word)
        return new_words

    def preprocessing(self,words):
        words = self.to_lowercase(words)
        words = self.replace_numbers(words)
        words = self.remove_punctuation(words)
        words = self.remove_non_ascii(words)
        words = self.remove_stopwords(words)
        return words
    def transform(self, X, y=None):
        X_ = X.copy()
        X_add = X_["study_and_condition"].str.split('.', 1, expand=True)
        X_add.columns = ['study', 'condition']
        X_ = pd.concat([X_, X_add], axis=1)
        X_.drop('study_and_condition', axis=1, inplace=True)
        X_['study'] = X_['study'].map(self.limpiar_study)
        X_['condition'] = X_['condition'].apply(contractions.fix) #Aplica la corrección de las contracciones
        X_['words'] = X_['condition'].apply(word_tokenize).apply(self.preprocessing) #Aplica la eliminación del ruido
        return X_

El segundo transformer, llamado NormalizacionTransformer, se encarga de realizar el stemming y lematización de las palabras en la entrada de tal manera que las palabras puedan ser relacionables en el modelo.

In [ ]:
class NormalizacionTransformer(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def stem_words(self,words):
        """Stem words in list of tokenized words"""
        stemmer = LancasterStemmer()
        stems = []
        for word in words:
            stem = stemmer.stem(word)
            stems.append(stem)
        return stems

    def lemmatize_verbs(self,words):
        """Lemmatize verbs in list of tokenized words"""
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        return lemmas

    def stem_and_lemmatize(self,words):
        stems = self.stem_words(words)
        lemmas = self.lemmatize_verbs(words)
        return stems + lemmas

    def transform(self, X, y=None):
        X_ = X.copy()
        X_['words'] = X_['words'].apply(self.stem_and_lemmatize) #Aplica lematización y Eliminación de Prefijos y Sufijos.
        X_['words'] = X_['words'].apply(lambda x: ' '.join(map(str, x)))
        return X_

# Pipeline
Una vez definidos los transformadores, se creó un pipeline que ejecuta primero la limpieza y luego la normalización. Después, este pipeline se exportó mediante joblib como un pickle para propósitos de la conexión con el API desarrollado.

In [ ]:
pipe = Pipeline(steps=[('limpieza', LimpiezaTransformer()),
                       ('normalizar', NormalizacionTransformer())])

joblib.dump(pipe, 'pipeline1.joblib')

# Uso en ambiente de producción
Una vez exportado el pipeline, también se tuvo que exportar el vectorizador y el modelo usados en la entrega previa. En este caso, el vectorizador tiene un rol muy importante dado que permite que un conjunto de palabras sean transformadas en un vector de la misma forma que los usados en el entrenamiento del modelo.

In [ ]:
pipe1=load('pipeline1.joblib')
vectorizer=load('vectorizer.joblib')
model=load('svcmodel.joblib')

Para poder predecir la elegibilidad de un nuevo registro se puede usar una función como la siguiente:

In [ ]:
def eligibility(register):
    registrotrans = pipe1.transform(register)
    registrotrans = vectorizer.transform(registrotrans['words'])
    prediction = model.predict(registrotrans)[0]
    proba = model.predict_proba(registrotrans)[0]
    if prediction:
        proba = round(proba[1],3)
    else:
        proba = round(proba[0],3)
    return ['Eligible' if prediction == 1 else 'Not eligible',proba]

Si se tiene el texto de un registro en una variable, en este caso study, se puede probar de la siguiente manera:

In [ ]:
#Probar la elegibilidad de un registro nuevo
eligibility(pd.DataFrame({'study_and_condition':[study]}))